In [1]:
# Versión 2.0
import pandas as pd
import numpy as np
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, BaggingRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_log_error, mean_absolute_percentage_error

## 1. Definimos nuestro problema

Dados dos dataset (train y test) con información de ventas de inmuebles, desarrollar un modelo a
partir del dataset train que permita predecir el valor aproximado/objetivo de los inmuebles en
el dataset test.

## 2. Construcción de nuestro dataset

*Consideraciones*:<br>
- Ambos datasets provistos constan de 80 features, pero test no contiene nuestra target feature, por lo que todos los steps de entrenamiento, validación y testeo lo haremos con train.<br>
- Test sólo será usado al final del proceso para el step de inference.<br>
- Train tiene 1460 registros, test tiene 1459.

In [2]:
# Importamos el dataset train, explicitamos 'MSSubClass' como variable categórica.
df = pd.read_csv('./Housing Dreams/house_train_raw.csv', dtype={'MSSubClass':str})
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
# Guardamos nuestro target en una variable y lo dropeamos junto con 'Id' del dataset.
target = df['SalePrice']
df = df.drop(['Id', 'SalePrice'], axis=1)

### 2.a. Pre-processing.

En ésta etapa crearemos pipelines para automatizar la limpieza y pre-procesamiento de los datos.<br>
*Consideraciones*:<br>
- En esta primera implementación me enfocaré en usar modelos 'tree-based' como pueden ser RandomForest o GradientBoosting, ya que no requieren de transformaciones adicionales (Scaling, OneHotEncoding) para performar adecuadamente.

Comenzamos por diferenciar las variables categóricas de las númericas.

In [4]:
cat_selector = make_column_selector(dtype_include=object)
num_selector = make_column_selector(dtype_include=np.number)
cat_selector(df)

['MSSubClass',
 'MSZoning',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'SaleType',
 'SaleCondition']

In [5]:
num_selector(df)

['LotFrontage',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageYrBlt',
 'GarageCars',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold']

El siguiente paso sería definir e implementar los pipelines de pre-procesamiento, cómo ya dijimos anteriormente, en ésta ocasión nos enfocaremos en modelos 'tree-based'. Por lo que nuestro pre-procesamiento se basará en imputar nulos y realizar ordinal encoding a las features categóricas.

In [6]:
cat_processor = OrdinalEncoder(
    handle_unknown='use_encoded_value', unknown_value=-1, encoded_missing_value=-1)

num_processor = SimpleImputer(strategy='median', add_indicator=False)

preprocessor = make_column_transformer(
    (num_processor, num_selector), (cat_processor, cat_selector)
)
preprocessor

ColumnTransformer(transformers=[('simpleimputer',
                                 SimpleImputer(strategy='median'),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x000001B23B4BBCA0>),
                                ('ordinalencoder',
                                 OrdinalEncoder(encoded_missing_value=-1,
                                                handle_unknown='use_encoded_value',
                                                unknown_value=-1),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x000001B22A1F3640>)])

### 2.b. Feature Engineering.

En ésta etapa crearemos un pipeline que aplique *SelectKBest* para elegir las 20 features más performantes para nuestros modelos 'tree-based'.

In [7]:
fe_pipeline = make_pipeline(
    preprocessor,
    SelectKBest(score_func=f_regression, k=20)
)
fe_pipeline.fit(df, target)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('simpleimputer',
                                                  SimpleImputer(strategy='median'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001B23B4BBCA0>),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(encoded_missing_value=-1,
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001B22A1F3640>)])),
                ('selectkbest',
                 SelectKBest(k=20,
                             score_func=<function f_regression at 0x000001B23BAD32E0>))])

Guardamos los nombres de las features elegidas para aplicarlo más tarde en el dataset 'Test' y aplicamos el filtro al df actual.

In [8]:
features_used = fe_pipeline.get_feature_names_out()
features_used = [feature.split('__')[1] for feature in features_used]
df1 = df[features_used]

## 3. Entrenamiento de los modelos.

Siguiendo la misma línea que en los pasos anteriores, se eligirán los 3 modelos más performantes para implementar las pipelines entre los siguentes modelos 'tree-based':<br>
- RandomForestRegressor.<br>
- GradientBoostingRegressor.<br>
- AdaBoostRegressor.<br>
- BaggingRegressor.

### 3.a. Hyperparameter Tuning

Antes de proceder con la implementación de nuestros pipelines tenemos que encontrar los mejores hiperparámetros para nuestros modelos.

In [9]:
# Primero sometemos nuestro df al pre-processing
df2 = preprocessor.fit_transform(df1)

In [10]:
# Dividimos nuestro df en 3:
# Train: 60% - Validation: 20% - Test: 20%
# En esta etapa sólo usaremos Train y Val, Test lo guardaremos para la siguiente etapa.

X_train, X_test1, y_train, y_test1 = train_test_split(df2, target, test_size=0.2, random_state=420)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=420)

Ahora tenemos que ir modelo por modelo buscando los mejores hiperparámetros, para ésto iteramos para cada modelo los siguientes pasos: <br>
1. Implementaremos el modelo con los hiperparámetros por defecto.<br>
2. Usaremos *RamdomizedSearch* para acotar nuestro rango de búsqueda.<br>
3. Finalmente ejecutaremos *GridSearch* para tratar de obtener la mejor combinación posible.

RandomForest:

In [11]:
# 1.
# Instanciamos y 'fiteamos' el modelo.
default_rf = RandomForestRegressor(random_state=420)
default_rf.fit(X_train, y_train)
# Hacemos prediciones usando el set de validación e imprimimos el MAPE.
y_pred = default_rf.predict(X_val)
mean_absolute_percentage_error(y_val, y_pred)  # 0.10426052369018328

0.10426052369018328

In [12]:
# 2.
# Seleccionaremos los hiperparámetros más importantes del modelo que estemos evaluando, tanto
# esto, como el rango de búsqueda los definimos buscando en Google.
param_dist = {
    'n_estimators': [*range(50, 301, 10)],
    'min_samples_leaf': [*range(1, 51)],
    'max_depth': [*range(2, 21)],
    'max_features': [1.0, 'sqrt'],
    'bootstrap': [True, False]}
# Especificamos el número de iteraciones que queremos que haga RandomizedSearch.
n_iter = 100
# Instanciamos y 'fiteamos' RandomizedSearch con los parámetros especificados.
rf_random_search = RandomizedSearchCV(
    estimator=default_rf,
    param_distributions=param_dist,
    n_iter=n_iter)
rf_random_search.fit(X_train, y_train)
# Hacemos prediciones usando el set de validación e imprimimos el MAPE.
y_pred = rf_random_search.predict(X_val)
mean_absolute_percentage_error(y_val, y_pred)  # 0.10252852408366316

RandomizedSearchCV(estimator=RandomForestRegressor(random_state=420),
                   n_iter=100,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [2, 3, 4, 5, 6, 7, 8, 9,
                                                      10, 11, 12, 13, 14, 15,
                                                      16, 17, 18, 19, 20],
                                        'max_features': [1.0, 'sqrt'],
                                        'min_samples_leaf': [1, 2, 3, 4, 5, 6,
                                                             7, 8, 9, 10, 11,
                                                             12, 13, 14, 15, 16,
                                                             17, 18, 19, 20, 21,
                                                             22, 23, 24, 25, 26,
                                                             27, 28, 29, 30, ...],
                                        'n_estimators': [50, 60, 70, 80, 90,
                                                         100, 110, 120, 130,
                                                         140, 150, 160, 170,
                                                         180, 190, 200, 210,
                                                         220, 230, 240, 250,
                                                         260, 270, 280, 290,
                                                         300]})

In [14]:
# Imprimimos los mejores parámetros del RandomizedSearch.
rf_random_search.best_params_

{'n_estimators': 210,
 'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'max_depth': 14,
 'bootstrap': False}

In [19]:
# 3.
# Basado en el resultado del RandomizedSearch definimos la grilla, usando valores cercanos.
param_grid = {
    'n_estimators': [190, 210, 230],
    'min_samples_leaf': [1, 2, 3],
    'max_depth': [10, 12, 14, 16, 18],
    'max_features': ['sqrt'],
    'bootstrap': [True, False]}
# Instanciamos y 'fiteamos' GridSearchCV usando la grilla definida, también utilizamos
# cross-validation para evitar el overfitting.
rf_gridsearch = GridSearchCV(
    estimator=default_rf,
    param_grid=param_grid,
    scoring='neg_mean_absolute_percentage_error',
    n_jobs=3,
    cv=5,
    refit=True,
    return_train_score=True)
rf_gridsearch.fit(X_train, y_train)
# Hacemos prediciones usando el set de validación e imprimimos el MAPE.
y_pred = rf_gridsearch.predict(X_val)
mean_absolute_percentage_error(y_val, y_pred)  # 0.10271417806379753

0.10271417806379753

Aunque el error sea un poco más alto que el que nos dió con RandomSearch igualmente utilizaremos el de GridSearch ya que este realizó el proceso de Cross-validation.

In [20]:
# Imprimimos los mejores parámetros del GridSearch, que serán los que usaremos en nuestro pipeline.
rf_gridsearch.best_params_

{'bootstrap': False,
 'max_depth': 16,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'n_estimators': 190}

GradientBoosting

In [21]:
# 1.
# Instanciamos y 'fiteamos' el modelo.
default_gb = GradientBoostingRegressor(random_state=420)
default_gb.fit(X_train, y_train)
# Hacemos prediciones usando el set de validación e imprimimos el MAPE.
y_pred = default_gb.predict(X_val)
mean_absolute_percentage_error(y_val, y_pred)  # 0.09823839869498618

0.09823839869498618

In [22]:
# 2.
# Seleccionaremos los hiperparámetros más importantes del modelo que estemos evaluando, tanto
# esto, como el rango de búsqueda los definimos buscando en Google.
param_dist = {
    'n_estimators': [*range(50, 301, 10)],
    'min_samples_leaf': [*range(1, 51)],
    'max_depth': [*range(2, 21)],
    'max_features': ['sqrt'],
    'subsample': [n * 0.1 for n in [*range(5, 11)]],
    'learning_rate': [n * 0.01 for n in [*range(5, 21)]],
    'min_samples_split': [*range(2, 21, 2)]
}
# Especificamos el número de iteraciones que queremos que haga RandomizedSearch.
n_iter = 300
# Instanciamos y 'fiteamos' RandomizedSearch con los parámetros especificados.
gb_random_search = RandomizedSearchCV(
    estimator=default_gb,
    param_distributions=param_dist,
    n_iter=n_iter)
gb_random_search.fit(X_train, y_train)
# Hacemos prediciones usando el set de validación e imprimimos el MAPE.
y_pred = gb_random_search.predict(X_val)
mean_absolute_percentage_error(y_val, y_pred)  # 0.10328280032059312

0.10328280032059312

In [23]:
# Imprimimos los mejores parámetros del RandomizedSearch.
gb_random_search.best_params_

{'subsample': 1.0,
 'n_estimators': 250,
 'min_samples_split': 14,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 8,
 'learning_rate': 0.05}

Como este es menos performante que el default, nos enfocaremos más en valores cercanos a los default para hacer el GridSearch

In [28]:
# 3.
# Basado en el resultado del RandomizedSearch y los valores default definimos la grilla.
param_grid = {
    'n_estimators': [50, 100, 150],
    'min_samples_leaf': [1, 2, 3],
    'max_depth': [3, 6, 9],
    'max_features': ['sqrt'],
    'subsample': [0.8, 0.9, 1.0],
    'learning_rate': [0.02, 0.05, 0.1],
    'min_samples_split': [2, 8, 14]
}
# Instanciamos y 'fiteamos' GridSearchCV usando la grilla definida, también utilizamos
# cross-validation para evitar el overfitting.
gb_gridsearch = GridSearchCV(
    estimator=default_gb,
    param_grid=param_grid,
    scoring='neg_mean_absolute_percentage_error',
    n_jobs=3,
    cv=4,
    refit=True,
    return_train_score=True)
gb_gridsearch.fit(X_train, y_train)
# Hacemos prediciones usando el set de validación e imprimimos el MAPE.
y_pred = gb_gridsearch.predict(X_val)
mean_absolute_percentage_error(y_val, y_pred)  # 0.10043094371795294

0.10043094371795294

Si bien este error dio mayor al default, tenemos que tener en cuenta que la configuración default también era posible y quedó por debajo de esta, la diferencia está en que en este paso hicimos Cross-Validation mientras que en el primer calculo no. Por lo tanto, tomaremos esta configuración de hiperparámetros.

In [29]:
# Imprimimos los mejores parámetros del GridSearch, que serán los que usaremos en nuestro pipeline.
gb_gridsearch.best_params_

{'learning_rate': 0.05,
 'max_depth': 6,
 'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 150,
 'subsample': 0.8}

AdaBoost.

In [11]:
# 1.
# Instanciamos y 'fiteamos' el modelo.
default_ab = AdaBoostRegressor(random_state=420)
default_ab.fit(X_train, y_train)
# Hacemos prediciones usando el set de validación e imprimimos el MAPE.
y_pred = default_ab.predict(X_val)
mean_absolute_percentage_error(y_val, y_pred)  # 0.1680321955730541

0.1680321955730541

In [12]:
# 2.
# Seleccionaremos los hiperparámetros más importantes del modelo que estemos evaluando, tanto
# esto, como el rango de búsqueda los definimos buscando en Google.
param_dist = {
    'n_estimators': [*range(10, 151, 10)],
    'learning_rate': [n * 0.01 for n in [*range(1, 21)]],
    'loss': ['linear', 'square', 'exponential']
}
# Especificamos el número de iteraciones que queremos que haga RandomizedSearch.
n_iter = 50
# Instanciamos y 'fiteamos' RandomizedSearch con los parámetros especificados.
ab_random_search = RandomizedSearchCV(
    estimator=default_ab,
    param_distributions=param_dist,
    n_iter=n_iter)
ab_random_search.fit(X_train, y_train)
# Hacemos prediciones usando el set de validación e imprimimos el MAPE.
y_pred = ab_random_search.predict(X_val)
mean_absolute_percentage_error(y_val, y_pred)  # 0.1524979866919682

0.1524979866919682

In [13]:
# Imprimimos los mejores parámetros del RandomizedSearch.
ab_random_search.best_params_

{'n_estimators': 150, 'loss': 'linear', 'learning_rate': 0.13}

In [14]:
# 3.
# Basado en el resultado del RandomizedSearch definimos la grilla.
param_grid = {
    'n_estimators': [100, 125, 150],
    'learning_rate': [0.07, 0.1, 0.13]
}
# Instanciamos y 'fiteamos' GridSearchCV usando la grilla definida, también utilizamos
# cross-validation para evitar el overfitting.
ab_gridsearch = GridSearchCV(
    estimator=default_ab,
    param_grid=param_grid,
    scoring='neg_mean_absolute_percentage_error',
    n_jobs=3,
    cv=5,
    refit=True,
    return_train_score=True)
ab_gridsearch.fit(X_train, y_train)
# Hacemos prediciones usando el set de validación e imprimimos el MAPE.
y_pred = ab_gridsearch.predict(X_val)
mean_absolute_percentage_error(y_val, y_pred)

0.1524979866919682

In [15]:
# Imprimimos los mejores parámetros del GridSearch, que serán los que usaremos en nuestro pipeline.
ab_gridsearch.best_params_

{'learning_rate': 0.13, 'n_estimators': 150}

Bagging.

In [16]:
# 1.
# Instanciamos y 'fiteamos' el modelo.
default_bag = BaggingRegressor(random_state=420)
default_bag.fit(X_train, y_train)
# Hacemos prediciones usando el set de validación e imprimimos el MAPE.
y_pred = default_bag.predict(X_val)
mean_absolute_percentage_error(y_val, y_pred)

0.107809109188161

In [18]:
# 2.
# Seleccionaremos los hiperparámetros más importantes del modelo que estemos evaluando, tanto
# esto, como el rango de búsqueda los definimos buscando en Google.
param_dist = {
    'n_estimators': [*range(10, 151, 10)],
    'bootstrap': [True, False]
}
# Especificamos el número de iteraciones que queremos que haga RandomizedSearch.
n_iter = 20
# Instanciamos y 'fiteamos' RandomizedSearch con los parámetros especificados.
bag_random_search = RandomizedSearchCV(
    estimator=default_bag,
    param_distributions=param_dist,
    n_iter=n_iter)
bag_random_search.fit(X_train, y_train)
# Hacemos prediciones usando el set de validación e imprimimos el MAPE.
y_pred = bag_random_search.predict(X_val)
mean_absolute_percentage_error(y_val, y_pred) #

0.10422084925509055

In [19]:
# Imprimimos los mejores parámetros del RandomizedSearch.
bag_random_search.best_params_

{'n_estimators': 100, 'bootstrap': True}

In [20]:
# 3.
# Basado en el resultado del RandomizedSearch definimos la grilla.
param_grid = {
    'n_estimators': [80, 90, 100, 110, 120],
    'bootstrap': [True, False]
}
# Instanciamos y 'fiteamos' GridSearchCV usando la grilla definida, también utilizamos
# cross-validation para evitar el overfitting.
bag_gridsearch = GridSearchCV(
    estimator=default_bag,
    param_grid=param_grid,
    scoring='neg_mean_absolute_percentage_error',
    n_jobs=3,
    cv=5,
    refit=True,
    return_train_score=True)
bag_gridsearch.fit(X_train, y_train)
# Hacemos prediciones usando el set de validación e imprimimos el MAPE.
y_pred = bag_gridsearch.predict(X_val)
mean_absolute_percentage_error(y_val, y_pred) # 0.1043602564003323

0.1043602564003323

In [21]:
# Imprimimos los mejores parámetros del GridSearch, que serán los que usaremos en nuestro pipeline.
bag_gridsearch.best_params_

{'bootstrap': True, 'n_estimators': 120}

### 3.b. Implementación de pipelines.

En este paso implementaremos las pipelines de los 3 modelos que mejor performaron en la etapa anterior:<br>
- RandomForest.<br>
- GradientBoosting.<br>
- Bagging.

In [22]:
rf_pipeline = make_pipeline(
    preprocessor,
    RandomForestRegressor(
        random_state=420,
        bootstrap=False,
        max_depth=16,
        max_features='sqrt',
        min_samples_leaf=1,
        n_estimators=190
    )
)
rf_pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('simpleimputer',
                                                  SimpleImputer(strategy='median'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001B23B4BBCA0>),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(encoded_missing_value=-1,
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001B22A1F3640>)])),
                ('randomforestregressor',
                 RandomForestRegressor(bootstrap=False, max_depth=16,
                                       max_features='sqrt', n_estimators=190,
                                       random_state=420))])

In [23]:
gb_pipeline = make_pipeline(
    preprocessor,
    GradientBoostingRegressor(
        random_state=420,
        learning_rate=0.05,
        max_depth=6,
        max_features='sqrt',
        min_samples_leaf=2,
        min_samples_split=2,
        n_estimators=150,
        subsample=0.8
    )
)
gb_pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('simpleimputer',
                                                  SimpleImputer(strategy='median'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001B23B4BBCA0>),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(encoded_missing_value=-1,
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001B22A1F3640>)])),
                ('gradientboostingregressor',
                 GradientBoostingRegressor(learning_rate=0.05, max_depth=6,
                                           max_features='sqrt',
                                           min_samples_leaf=2, n_estimators=150,
                                           random_state=420, subsample=0.8))])

In [24]:
bag_pipeline = make_pipeline(
    preprocessor,
    BaggingRegressor(random_state=420, bootstrap=True, n_estimators=120)
)
bag_pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('simpleimputer',
                                                  SimpleImputer(strategy='median'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001B23B4BBCA0>),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(encoded_missing_value=-1,
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001B22A1F3640>)])),
                ('baggingregressor',
                 BaggingRegressor(n_estimators=120, random_state=420))])

### 3.c. Model training usando los pipelines.

*Consideraciones*:<br>
- La validación ya fue realizada mientras haciamos el Hyperparameter Tuning. Por lo tanto ahora realizaremos el split del set 'train' original nuevamente en 2 sets, Train y Test.

In [31]:
# Dividimos nuestro df original en 2:
# Train: 80% - Test: 20%

X_train, X_test, y_train, y_test = train_test_split(df1, target, test_size=0.2, random_state=420)

In [32]:
# Entrenamos los modelos.
rf_pipeline.fit(X_train, y_train)

gb_pipeline.fit(X_train, y_train)

bag_pipeline.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('simpleimputer',
                                                  SimpleImputer(strategy='median'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001B23B4BBCA0>),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(encoded_missing_value=-1,
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001B22A1F3640>)])),
                ('baggingregressor',
                 BaggingRegressor(n_estimators=120, random_state=420))])

## 4. Evaluación de los modelos.

In [33]:
# Realizamos todas las predicciones usando X_test.
y_preds_rf = rf_pipeline.predict(X_test)

y_preds_gb = gb_pipeline.predict(X_test)

y_preds_bag = bag_pipeline.predict(X_test)

In [34]:
# Calculamos la métrica requerida para cada uno de nuestros modelos
l_func_rf = mean_squared_log_error(y_test, y_preds_rf, squared=False)
l_func_gb = mean_squared_log_error(y_test, y_preds_gb, squared=False)
l_func_bag = mean_squared_log_error(y_test, y_preds_bag, squared=False)
print(f'Random Forest Reg: {l_func_rf}')
print(f'Gradient Boosting Reg: {l_func_gb}')  # Winner
print(f'Bagging Reg: {l_func_bag}')

Random Forest Reg: 0.15830764137232117
Gradient Boosting Reg: 0.15162142576251303
Bagging Reg: 0.15831734983629336


In [36]:
# Calculamos el MAPE del modelo que mejor performó (GradientBoostingReg)
l_func_gb = mean_absolute_percentage_error(y_test, y_preds_gb)*100
l_func_gb  # 89.5% de accuracy

10.517167020160423

## 5. Model Inference.

En ésta etapa haremos las predicciones de nuestro segundo dataset y guardaremos las mismas en un formato CSV.

In [37]:
# Lectura del Dataset.
df_inf = pd.read_csv('./Housing Dreams/houses_test_raw.csv')
id_inf = df_inf['Id']

In [38]:
# Filtramos por las features elegidas usando SelectKBest.
X_inf = df_inf[features_used]

In [39]:
# Realizamos nuestras predicciones!.
y_inf = gb_pipeline.predict(X_inf)

In [40]:
# Acá transformamos nuestras predicciones de np.array a pd.DataFrame para poder usar el método .to_csv() para guardar nuestras predicciones en el formato requerido.
y_inf_df = pd.DataFrame(y_inf, index=id_inf, columns=['pred'])
y_inf_df.to_csv('pred_test.csv', index=True)